In [1]:
    # from selenium import webdriver
# import time
# from selenium.webdriver.support.ui import WebDriverWait
# from selenium.webdriver.support import expected_conditions as EC
# from selenium.webdriver.support.ui import Select
# from selenium.webdriver.common.by import By
# import os

In [2]:
# Import relevant libraries
from splinter import Browser
from bs4 import BeautifulSoup
import os

In [3]:
DATA_DIR = "data"
STANDINGS_DIR = os.path.join(DATA_DIR, "standings")
SCORES_DIR = os.path.join(DATA_DIR, "scores")

In [4]:
seasons = list(range(2021,2024))
seasons

[2021, 2022, 2023]

In [5]:
# Set up the browser
executable_path = {'executable_path':"C:\Program Files (x86)\msedgedriver.exe"}
browser = Browser('edge', **executable_path)

In [6]:
for season in seasons:
    # Visit the website for scraping
    url = f"https://www.basketball-reference.com/leagues/NBA_{season}_games.html"
    browser.visit(url)

    # Create a BeautifulSoup object
    html = browser.html
    soup = BeautifulSoup(html, 'html.parser')

    filter_div = soup.find('div', class_='filter')
    links = filter_div.find_all('a')
    urls = [link.get("href") for link in links]

In [7]:
table = soup.find('table', id='schedule')

links = table.find_all("a")
hrefs = [link.get('href') for link in links]
box_scores = [link for link in hrefs if link and "boxscore" in link and ".html" in link]
box_scores = [f"https://www.basketball-reference.com{score}" for score in box_scores]

In [84]:
box_score = box_scores[0]

browser.visit(box_score)

html=browser.html
soup = BeautifulSoup(html, 'html.parser')

In [85]:
score_table = soup.find('table', id='line_score')
tbody = score_table.find('tbody')
rows = tbody.find_all('tr')

line_score = []

for row in rows:
    # Get team name
    team = row.find('th', class_='center').text
    columns = row.find_all('td')
    
    # Create line score dictionary
    if(columns !=[]):
        total = columns[4].text
    
    # Create dictionary for dataframe later
    line_score_dict = { "team": team,
                        "total": total
    }
    
    # Add dictionary to array
    line_score.append(line_score_dict)

In [86]:
# Create Data frame
import pandas as pd

df = pd.DataFrame(line_score)
df

,team,total
0,PHI,117
1,BOS,126


In [121]:
# Get basic and advanced box scores
teams = []
for i in range(len(line_score)):
    teams.append(line_score[i]['team'])

In [274]:
summaries = []
basic = []
advanced = []

# add 'advanced' after
stats = ['basic']
for team in teams:
    for stat in stats:
        if stat == 'basic':
            stat_table = soup.find('table', id=f"box-{team}-game-{stat}")
            tbody = stat_table.find('tbody')
            thead = stat_table.find('thead')
            
            # Find headers in table 
            headers = [header.text for header in thead.find_all('tr')[1].find_all('th')]
            
            # Find rows of table body
            rows = tbody.find_all('tr')
            basic = []
            
            # Loop through rows for data
            for row in rows:
                player_link = row.find('a')
                if player_link:
                    player = row.find('th').text
                    columns = row.find_all('td')
                    
                    if len(columns) >=2:
                        
                    # Create basic stat dictionary
                        MP = columns[0].text
                        FG = columns[1].text
                        FGA = columns[2].text
                        FG_pct = columns[3].text
                        three_pts = columns[4].text
                        three_pt_att = columns[5].text
                        three_pt_pct = columns[6].text
                        FT = columns[7].text
                        FTA = columns[8].text
                        FT_pct = columns[9].text
                        ORB = columns[10].text
                        DRB = columns[11].text
                        TRB = columns[12].text
                        AST = columns[13].text
                        STL = columns[14].text
                        BLK = columns[15].text
                        TOV = columns[16].text
                        PF = columns[17].text
                        PTS = columns[18].text
                        DIFF = columns[19].text
                        
                # Create dictionary for dataframe later
                basic_dict = { "player": player,
                                "MP": MP,
                                "FG": FG,
                                "FGA": FGA,
                              "FG%": FG_pct,
                              "3P": three_pts,
                              "3PA": three_pt_att,
                              "3P%" :three_pt_pct,
                              "FT": FT,
                              "FTA": FTA,
                              "FT%": FT_pct,
                              "ORB": ORB,
                              "DRB": DRB,
                              "TRB": TRB,
                              "AST": AST,
                              "STL": STL,
                              "BLK": BLK,
                              "TOV": TOV,
                              "PF": PF,
                              "PTS": PTS,
                              "+/-" : DIFF
                }

                # Add dictionary to array
                basic.append(basic_dict)

In [273]:
# Create Data frame
import pandas as pd

df = pd.DataFrame(basic)
df

,player,MP,FG,FGA,FG%,3P,3PA,3P%,FT,FTA,...,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS,+/-
0,Jayson Tatum,38:38,13,20,.650,2,7,.286,7,9,...,0,12,12,4,1,1,3,1,35,+10
1,Jaylen Brown,38:34,14,24,.583,4,12,.333,3,3,...,0,3,3,2,2,1,4,1,35,+7
2,Marcus Smart,36:05,3,8,.375,1,4,.250,7,8,...,1,2,3,7,1,0,1,3,14,0
3,Derrick White,24:02,1,3,.333,0,1,.000,0,1,...,1,2,3,4,1,0,1,2,2,+9
4,Al Horford,23:06,2,7,.286,2,5,.400,0,0,...,1,4,5,1,0,0,0,4,6,+8
5,Al Horford,23:06,2,7,.286,2,5,.400,0,0,...,1,4,5,1,0,0,0,4,6,+8
6,Grant Williams,23:57,5,5,1.000,3,3,1.000,2,3,...,0,1,1,1,0,0,0,3,15,-3
7,Malcolm Brogdon,23:52,7,11,.636,0,2,.000,2,2,...,1,1,2,4,2,0,1,2,16,+9
8,Noah Vonleh,20:10,1,2,.500,0,0,,0,0,...,0,2,2,0,1,1,0,4,2,+7
9,Blake Griffin,8:17,0,2,.000,0,1,.000,1,2,...,2,3,5,1,0,0,0,3,1,-5


In [12]:
# for url in box_scores:
#     save_path = os.path.join(SCORES_DIR, url.split("/")[-1])
#     if os.path.exists(save_path):
#         continue
    
#     browser.visit(url)
#     # Get html from url links
#     html = browser.html
#     if html:
#         soup = BeautifulSoup(html, "html.parser")
#         content = soup.find('div', id='content')
#         if conbasic_dict#             content_text = content.text
       
#         # Save to folder
#         with open(save_path, "w+", encoding='utf-8') as f:
#             f.write(content_text)

In [13]:
# standings_files

In [14]:
# files

In [15]:
# DATA_DIR = "data"
# STANDINGS_DIR = os.path.join(DATA_DIR, "standings")
# SCORES_DIR = os.path.join(DATA_DIR, "scores")